# 네이버 뉴스검색

In [1]:
import requests
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import os

In [2]:
date = str(datetime.now())
print(date)
date = date[:date.rfind(':')].replace(' ', '_')
print(date)
date = date.replace(':', '시') + '분'
print(date)

2022-09-06 13:13:36.260903
2022-09-06_13:13
2022-09-06_13시13분


In [3]:
query = input('검색 키워드를 입력하세요 : ')
news_num = int(input('총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : '))
query = query.replace(' ', '+')

news_url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'

req = requests.get(news_url.format(query))
soup = BeautifulSoup(req.text, 'html.parser')

검색 키워드를 입력하세요 : 태풍
총 필요한 뉴스기사 수를 입력해주세요(숫자만 입력) : 3000


In [4]:
news_dict = {}
idx = 0
cur_page = 1

print()
print('크롤링 중...')


크롤링 중...


In [5]:
while idx < news_num:
    ### 네이버 뉴스 웹페이지 구성이 바뀌어 태그명, class 속성 값 등을 수정함(20210126) ###

    table = soup.find('ul', {'class': 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class': 'news_area'}) for li in li_list]
    a_list = [area.find('a', {'class': 'news_tit'}) for area in area_list]

    for n in a_list[:min(len(a_list), news_num - idx)]:
        news_dict[idx] = {'title': n.get('title'),
                          'url': n.get('href')}
        idx += 1

    cur_page += 1

    pages = soup.find('div', {'class': 'sc_page_inner'})
    next_page_url = [p for p in pages.find_all('a') if p.text == str(cur_page)][0].get('href')

    req = requests.get('https://search.naver.com/search.naver' + next_page_url)
    soup = BeautifulSoup(req.text, 'html.parser')

print('크롤링 완료')

크롤링 완료


In [6]:
print('데이터프레임 변환')
news_df = DataFrame(news_dict).T

folder_path = os.getcwd()
xlsx_file_name = '네이버뉴스_{}_{}.xlsx'.format(query, date)

news_df.to_excel(xlsx_file_name)                 # 엑셀 파일로 저장

print('엑셀 저장 완료 | 경로 : {}\\{}'.format(folder_path, xlsx_file_name))
os.startfile(folder_path)

데이터프레임 변환
엑셀 저장 완료 | 경로 : C:\Users\giduc\python_analysis\crawling\네이버뉴스_태풍_2022-09-06_13시13분.xlsx


In [7]:
# 엑셀 파일을 읽어와서 result.txt 파일로 저장
df = pd.read_excel(xlsx_file_name)
# print(df)

title = df[['title']]
print(title)

title.to_csv('result.txt')                 # result.txt 파일로 저장

                                            title
0     [속보] 중대본 "태풍에 1명 사망·2명 실종"…시설물 파손·침수 피해도 속출
1           포스코 포항제철소 공장 2곳서 화재…"태풍 연관성 조사"(종합2보)
2              尹대통령 "태풍 완전히 지날 때까지 긴장의 끈 놓지 말아달라"
3                 포스코 포항제철소 공장 2곳서 화재…"태풍 연관성 조사"
4                  '태풍의 상처' 서울 도로 곳곳 통제…이젠 출근길 비상
...                                           ...
2995  제주·부산·경남 등 남부지방 6일 ‘등교 중지’ [‘괴물 태풍’ 힌남노 상륙]
2996   [B tv 서울뉴스] 태풍 피해 우려 '서울·인천 임시선별검사소' 운영 중단
2997               박완수 도지사, 태풍 ‘힌남노’ 대비 재해취약지역 점검
2998                                 태풍 '힌남노' 근접중
2999                      진병영 함양군수, 태풍 대비 현장점검 나서

[3000 rows x 1 columns]
